# NCU輔助選課程式，提供選課時的分發機率參考

## 使用方法
1. 拷貝這份程式到電腦或雲端硬碟(推薦)，以保存選課資料
2. 查找課堂的流水號在**選課系統**->**課程加退選**->**選課**->**志願序與選修別**，可搭配ncu course finder使用
3. 在下方`course_numbers`中輸入你想要的課程流水號，就可查詢該課程的分發機率(第1志願有多少機率會上、第2志願有多少機率會上...)。
4. 執行全部程式
5. 根據各個課堂上的機率，排課程的志願序 EX：已知A堂課只有在第1志願才會上，就不用把第2志願的名額給A堂課，可以給有可能在第2志願上的B堂課

## 分發順序
興趣選項課程分發規則如下：
1. 課程修課對象優先順序條件限制
2. 學生登錄之同一課群志願序(此程式算的是這個機率)
3. 系統自動產生之亂數籤號

## 範例結果
- **課程名稱**: 日文（一）A LN0025 - A
- **人數限制**: 50
- **中選人數**: 0
- **第1志願上的機率**: 16.61%
- **已分發完畢**

## 注意事項
1. 此程式只適用興趣選項課程(通識、體育、語言類課程等等)，且此程式預設每個人的優先順序都是相同，請先查看你的優先順序是否為第一
2. 執行太多次程式或輸入的課堂號過多可能會鎖IP位置，推薦在colab上執行且課堂號不要一次輸入過多

## 製作人員
資工二A 陳聖勳

## 其他問題
請聯繫 email:tom1030507@gmail.com

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

def get_course_detail(course_serial_number):
    # 定義目標URL
    url = 'https://cis.ncu.edu.tw/Course/main/support/courseDetail.html?crs=' + course_serial_number

    # 發送HTTP請求
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    def get_value_by_label(label):
        td = soup.find('td', class_='subTitle', string=label)
        if td:
            next_td = td.find_next_sibling('td')
            if next_td:
                # 排除不需要的部分
                for span in next_td.find_all('span', class_='engclass'):
                    span.decompose()
                return next_td.get_text(strip=True)
        return None

    course_name = get_value_by_label('課程名稱/備註')
    course_code = get_value_by_label('流水號  / 課號')

    # 提取課號
    match = re.search(r'[A-Za-z0-9]+ - [A-Za-z*]', course_code)
    if match:
        course_code = match.group()
    else:
        print('未知課號\n')
        return

    enrollment_limit = get_value_by_label('人數限制')
    selected_count = get_value_by_label('中選人數')

    print(f'課程名稱: {course_name} {course_code}')
    print(f'人數限制: {enrollment_limit}')
    print(f'中選人數: {selected_count}')

    remaining_slots = int(enrollment_limit) - int(selected_count)

    # 查找特定內容出現的次數
    td_elements = soup.find_all('td')
    for priority in range(1, 10):
        count = 0
        for index in range(len(td_elements) - 1):
            if td_elements[index].get_text(strip=True) == str(priority) and td_elements[index + 1].get_text(strip=True) == '待分發':
                count += 1
        if count == 0:
            print(f'第{priority}志願上的機率: 100.00%')
            continue
        probability = remaining_slots / count * 100
        if probability > 100:
            probability = 100
        print(f'第{priority}志願上的機率: {probability:.2f}%')
        remaining_slots -= count
        if remaining_slots <= 0:
            print("已分發完畢")
            break
    print()

# 輸入你想要的課程流水號(4或5位號碼)
pe_course_numbers = ["1041", "1043"]
japanese_course_numbers = ["0001", "0002"]
general_course_numbers = ["9007", "9018"]

print("<體育課程>\n")
for serial_number in pe_course_numbers:
    get_course_detail(serial_number)

print("<日文課程>\n")
for serial_number in japanese_course_numbers:
    get_course_detail(serial_number)

print("<通識課程>\n")
for serial_number in general_course_numbers:
    get_course_detail(serial_number)

<體育課程>

課程名稱: 羽球入門 PE2101 - A
人數限制: 40
中選人數: 0
第1志願上的機率: 12.62%
已分發完畢

課程名稱: 羽球入門 PE2101 - C
人數限制: 40
中選人數: 0
第1志願上的機率: 14.76%
已分發完畢

<日文課程>

課程名稱: 日文(一)A LN0025 - A
人數限制: 50
中選人數: 0
第1志願上的機率: 15.11%
已分發完畢

課程名稱: 日文(一)A LN0025 - B
人數限制: 50
中選人數: 0
第1志願上的機率: 10.16%
已分發完畢

<通識課程>

課程名稱: 西方古典音樂欣賞 GS2153 - *
人數限制: 120
中選人數: 0
第1志願上的機率: 34.58%
已分發完畢

課程名稱: 行政法 GS3077 - *
人數限制: 130
中選人數: 0
第1志願上的機率: 100.00%
第2志願上的機率: 66.67%
已分發完畢

